In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_clirad.fillna('-', inplace=True)
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,mls,mls
band,[1],1
commitnumber,1013f91,5014a19
conc,NaN,None
dv,NaN,0.001
klin,NaN,0
molecule,{'h2o': 'atmpro'},h2o
ng_adju,NaN,"[0, 0, 0]"
ng_refs,NaN,"[3, 3, 6]"
nv,NaN,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  6.538142                    1  3.931834
0.000750        2  5.055377                    2  3.666955
0.001052        3  4.528686                    3  3.533896
0.001476        4  3.991609                    4  3.347235
0.002070        5  3.463416                    5  3.095256
0.002904        6  2.964688                    6  2.775839
0.004074        7  2.507539                    7  2.398476
0.005714        8  2.095885                    8  1.990339
0.008015        9  1.729136                    9  1.594670
0.011243       10  1.406249                   10  1.260706
0.015771       11  1.126271                   11  1.016532
0.022122       12  0.886746                   12  0.846305
0.031031       13  0.682595                   13  0.706839
0.043528       14  0.498515                   14  0.553509
0.061057       15  0.368263                   15  0.412955
0.085645       16  0.349431                   16  0.356650
0.120136       17  0.375653                   17  0.349504
0.168516       18  0.385030                   18  0.349901
0.236378       19  0.396302                   19  0.371387
0.331549       20  0.418323                   20  0.406083
0.465100       21  0.457010                   21  0.445698
0.652400       22  0.524987                   22  0.510031
0.915100       23  0.606909                   23  0.610816
1.283650       24  0.618638                   24  0.663259
1.800600       25  0.570939                   25  0.642822
2.525700       26  0.529225                   26  0.588971
3.542800       27  0.494019                   27  0.534225
4.969550       28  0.461344                   28  0.467433
6.970850       29  0.432716                   29  0.418876
9.778100       30  0.406825                   30  0.398618
13.715850      31  0.378515                   31  0.374700
19.239350      32  0.345810                   32  0.338649
26.987250      33  0.318185                   33  0.319205
37.855300      34  0.298341                   34  0.301198
53.100050      35  0.275279                   35  0.272011
73.887500      36  0.247625                   36  0.238729
97.662500      37  0.228765                   37  0.210429
121.437500     38  0.218934                   38  0.196072
145.212500     39  0.201939                   39  0.181068
168.987500     40  0.192083                   40  0.175049
192.762500     41  0.298105                   41  0.279792
216.537500     42  0.719739                   42  0.692339
240.312500     43  1.142834                   43  1.121280
264.087500     44  1.371455                   44  1.425960
287.862500     45  1.367191                   45  1.383643
311.637500     46  1.090029                   46  1.105848
335.412500     47  0.874029                   47  0.930667
359.187500     48  0.728956                   48  0.755233
382.962500     49  0.610124                   49  0.605174
406.737500     50  0.501075                   50  0.502912
430.512500     51  0.397690                   51  0.416508
454.287500     52  0.297351                   52  0.312526
478.062500     53  0.212697                   53  0.209297
501.837500     54  0.146253                   54  0.125752
525.612500     55  0.096576                   55  0.069745
549.387500     56  0.061861                   56  0.037715
573.162500     57  0.039594                   57  0.022542
596.937500     58  0.026239                   58  0.015714
620.712500     59  0.016982                   59  0.011292
644.487500     60  0.011297                   60  0.008440
668.262500     61  0.008214                   61  0.006670
692.037500     62  0.006487                   62  0.005496
715.812500     63  0.005621                   63  0.005021
739.587500     64  0.005374                   64  0.005005
763.362500     65  0.005881                   65  0.005597
78

# Fluxes by Layer

CRD                           CLIRAD                \
                        flug       fldg      fnetg       flug          fldg   
pressure    level                                                             
0.000000    1     -34.263992   0.000000 -34.263992 -34.073599 -6.957992e-07   
0.000624    2     -34.264010   0.000501 -34.263509 -34.073615  3.066839e-04   
0.000876    3     -34.264017   0.000659 -34.263358 -34.073622  4.225607e-04   
0.001229    4     -34.264026   0.000858 -34.263169 -34.073630  5.788687e-04   
0.001723    5     -34.264038   0.001103 -34.262935 -34.073642  7.863011e-04   
0.002417    6     -34.264053   0.001403 -34.262650 -34.073656  1.055343e-03   
0.003391    7     -34.264073   0.001765 -34.262309 -34.073675  1.394377e-03   
0.004757    8     -34.264098   0.002196 -34.261903 -34.073700  1.807011e-03   
0.006672    9     -34.264131   0.002704 -34.261427 -34.073732  2.290844e-03   
0.009359    10    -34.264175   0.003298 -34.260877 -34.073774  2.840851e-03   
0.013128    11    -34.264233   0.003983 -34.260249 -34.073831  3.460436e-03   
0.018415    12    -34.264311   0.004767 -34.259544 -34.073909  4.174836e-03   
0.025830    13    -34.264421   0.005656 -34.258765 -34.074019  5.028248e-03   
0.036232    14    -34.264585   0.006661 -34.257924 -34.074182  6.062234e-03   
0.050823    15    -34.264862   0.007801 -34.257062 -34.074455  7.292048e-03   
0.071291    16    -34.265347   0.009178 -34.256169 -34.074921  8.759662e-03   
0.100000    17    -34.266043   0.011062 -34.254981 -34.075578  1.063003e-02   
0.140271    18    -34.266899   0.013710 -34.253188 -34.076373  1.309231e-02   
0.196760    19    -34.267930   0.017318 -34.250612 -34.077311  1.637165e-02   
0.275997    20    -34.269147   0.022254 -34.246893 -34.078384  2.093139e-02   
0.387100    21    -34.270515   0.029128 -34.241387 -34.079532  2.742390e-02   
0.543100    22    -34.271879   0.038938 -34.232942 -34.080552  3.668153e-02   
0.761700    23    -34.272540   0.053193 -34.219348 -34.080711  5.004856e-02   
1.068500    24    -34.270851   0.073560 -34.197291 -34.078383  6.992194e-02   
1.498800    25    -34.266117   0.100360 -34.165758 -34.072896  9.824637e-02   
2.102400    26    -34.259154   0.134219 -34.124935 -34.065017  1.363340e-01   
2.949000    27    -34.250006   0.178145 -34.071861 -34.054835  1.852236e-01   
4.136600    28    -34.238366   0.236003 -34.002363 -34.042024  2.475744e-01   
5.802500    29    -34.223948   0.312626 -33.911322 -34.026433  3.242353e-01   
8.139200    30    -34.206152   0.414606 -33.791546 -34.007364  4.211225e-01   
11.417000   31    -34.184037   0.550453 -33.633585 -33.984028  5.525778e-01   
16.014700   32    -34.157724   0.730289 -33.427434 -33.956576  7.292194e-01   
22.464000   33    -34.130450   0.967203 -33.163247 -33.928314  9.597003e-01   
31.510500   34    -34.104899   1.282626 -32.822273 -33.901990  1.275480e+00   
44.200100   35    -34.080035   1.706221 -32.373814 -33.876960  1.703249e+00   
62.000000   36    -34.062659   2.269277 -31.793382 -33.860567  2.260458e+00   
85.775000   37    -34.075578   2.979587 -31.095991 -33.876528  2.948826e+00   
109.550000  38    -34.117337   3.665620 -30.451716 -33.922074  3.587068e+00   
133.325000  39    -34.175205   4.340075 -29.835130 -33.984031  4.201285e+00   
157.100000  40    -34.261221   4.994817 -29.266404 -34.074572  4.801822e+00   
180.875000  41    -34.394933   5.669496 -28.725437 -34.212507  5.432802e+00   
204.650000  42    -34.597482   6.711603 -27.885878 -34.420503  6.428863e+00   
228.425000  43    -34.924059   9.065195 -25.858864 -34.755847  8.714256e+00   
252.200000  44    -35.366352  12.726073 -22.640279 -35.212894  1.232951e+01   
275.975000  45    -35.915643  17.137817 -18.777826 -35.778419  1.691141e+01   
299.750000  46    -36.564682  21.637301 -14.927381 -36.447946  2.147812e+01   
323.525000  47    -37.247988  25.390476 -11.857513 -37.150079  2.529499e+01   
347.300000  48    -37.940117  28.544147  -9.395970 -37.859307  2.862554e+01

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')